# ДОМАШНЯЯ РАБОТА - "рекомендации на основе скрытых факторов"

ДАНИЛЬЧЕНКО ВАДИМ

ЗАДАНИЕ:

“Требуется построить модель рекомендаций на основе скрытых факторов (implicit) на основе dataset’а https://grouplens.org/datasets/hetrec-2011/ (Delicious Bookmarks)”

-------------------------------------------------------------------------------------------------------------------------------

РЕШЕНИЕ:

в процессе взаимодействия с сервисом пользователь ставит определнные тэги, за каждым тэгом скрывается некоторый элемент (ссылка, товар, некоторый контент, который я не вижу). 

В данной работе я попробую рекомендовать тэги пользователям со схожими интересами

In [44]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import scipy.sparse as sparse

%matplotlib inline

from implicit.als import AlternatingLeastSquares

загрузим данные по пользователям и тэгам, которые они проставляли

In [2]:
data=pd.read_table('data/user_taggedbookmarks.dat')
data.head()

,userID,bookmarkID,tagID,day,month,year,hour,minute,second
0,8,1,1,8,11,2010,23,29,22
1,8,2,1,8,11,2010,23,25,59
2,8,7,1,8,11,2010,18,55,1
3,8,7,6,8,11,2010,18,55,1
4,8,7,7,8,11,2010,18,55,1


In [24]:
data.shape

(437593, 9)

In [28]:
tags=pd.read_table('data/tags.dat', encoding='cp1251')
tags.rename(columns={'id':'tagID'}, inplace=True)
tags.head()

,tagID,value
0,1,collection_development
1,2,library
2,3,collection
3,4,development
4,5,lesson_plan


In [26]:
tags.shape

(53388, 2)

In [30]:
data_joined=data.merge(tags, on='tagID', how='left')
data_joined.head()

,userID,bookmarkID,tagID,day,month,year,hour,minute,second,value
0,8,1,1,8,11,2010,23,29,22,collection_development
1,8,2,1,8,11,2010,23,25,59,collection_development
2,8,7,1,8,11,2010,18,55,1,collection_development
3,8,7,6,8,11,2010,18,55,1,war
4,8,7,7,8,11,2010,18,55,1,veterens


In [32]:
data2analyse=data_joined.groupby(['userID', 'tagID', 'value']).day.count().reset_index().rename(columns={'day':'cnt', 'value':'tag_name'})
data2analyse.head()

,userID,tagID,tag_name,cnt
0,8,1,collection_development,28
1,8,2,library,7
2,8,5,lesson_plan,1
3,8,6,war,1
4,8,7,veterens,1


In [33]:
data2analyse.shape

(230939, 4)

In [42]:
# список уникальных id тэгов
tag_name=tags.tagID.unique().tolist()
users=data2analyse.userID.unique().tolist()

In [67]:
len(tag_name)

53388

In [50]:
rows = data2analyse.userID.astype(int)
cols = data2analyse.tagID.astype(int)

построим модель

In [68]:
# создадим разреженную матрицу с id_пользоователя по строкам и id_артиста по столбцам
data_sparse = sparse.csr_matrix((data2analyse.cnt.tolist(), (cols,rows)))

In [69]:
# обучим модель
model = AlternatingLeastSquares(factors=50)
model.fit(data_sparse)

100%|████████████████████████████████████████████████████████████████████████████████| 15.0/15 [00:01<00:00,  7.79it/s]


сделаем рекомендацию элементов с такими тэгами пользователю с id=8

ps: т.к. знаю только тэги, не вывожу сами элементы, только тэги

In [70]:
userid = 8

user_items = data_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)

In [71]:
recommendations

[(208, 0.7285592),
 (195, 0.5801674),
 (209, 0.5663497),
 (640, 0.5245481),
 (1636, 0.5123596),
 (675, 0.5085158),
 (115, 0.50678337),
 (3, 0.49994874),
 (174, 0.4790666),
 (323, 0.4789387)]

In [79]:
for r in recommendations:
    print(tags[tags.tagID==r[0]].value.tolist()[0])

international
teaching
digital
rules
vnc
economy
insurance
collection
web20
money


посмотрим какие тэги в большинстве ставил пользователь с id=8

In [100]:
data2analyse[data2analyse.userID==8].sort_values(by='cnt', ascending=False).head(10)

,userID,tagID,tag_name,cnt
0,8,1,collection_development,28
36,8,45,media_awareness,8
1,8,2,library,7
20,8,24,librarian,6
55,8,76,internet_safety,4
21,8,25,literacy,4
60,8,82,students,4
49,8,66,learning,3
7,8,10,read_books,2
54,8,71,marc,2


найдем схожие тэги

In [101]:
itemid = 1
related = model.similar_items(itemid)

In [102]:
related

[(1, 0.98083866),
 (19, 0.86995834),
 (89, 0.8615656),
 (18, 0.8163761),
 (208, 0.8136119),
 (244, 0.8091136),
 (406, 0.8082222),
 (575, 0.80610234),
 (267, 0.8020966),
 (586, 0.8005501)]

In [103]:
for r in related:
    print(tags[tags.tagID==r[0]].value.tolist()[0])

collection_development
best_books_list
critical
fingertips
international
criticalthinking
options
propinquity
ipodtouch
dave-snowden
